In [ ]:
import pandas as pd
import requests
import sqlite3
import re
from bs4 import BeautifulSoup

In [143]:
pt_london_2025_decklists_url = "https://fabtcg.com/en/coverage/pro-tour-london/decklist/{}/"
pt_london_2025_results_url = "https://fabtcg.com/en/coverage/pro-tour-london/results/{}/"

In [144]:
rounds = [number + 1 for number in range(18) if not 5 <= number <= 11]

In [145]:
def get_pairings(url, rounds_to_be_considered):
    output = []

    for round in rounds_to_be_considered:
        page = requests.get(url.format(round))
        soup = BeautifulSoup(page.text, "html")

        player = soup.find_all("div", {"class":"tournament-coverage__player-hero-and-deck"})
        winner = soup.find_all("div", {"class":"tournament-coverage__result"})

        for idx, id in enumerate(player):
            table = int(idx / 2) + 1

            seat = (idx % 2) + 1

            gem_id = id.find("a", href = True)

            if gem_id:
                gem_id = re.search(r"\/(\d+)", gem_id["href"]).group(1)

            if re.search(r"(\d)", winner[table - 1].text):
                winning_seat = int(re.search(r"(\d)", winner[table - 1].text).group(1))
            else:
                print(f"Round: {round} Table: {table} had a draw")
                winning_seat = 0

            if winning_seat == 0:
                player_status = "Draw"
            elif winning_seat == seat:
                player_status = "Win"
            else:
                player_status = "Loss"

            record = [round, table, seat, gem_id, player_status]

            output.append(record)

    return output

def make_player_list(pairings):
    output = []

    for record in pairings:
        if record[3] not in output and record[3]:
            output.append(record[3])

    return output

def get_decklist(player_list, decklist_url):
    output = []

    for id in player_list:
        url = decklist_url.format(id)

        page = requests.get(url)
        soup = BeautifulSoup(page.text, "html")

        if page.status_code == 200:
            output.append([data.text.strip() for data in soup.find_all("td")])
        else:
            output.append([id, "Unknown", None, None, None, "Unknown"])

    return output

def make_hero_list(decklists):
    output = []

    for record in decklists:
        if record[4] not in output and record[4]:
            output.append(record[4])

    return output

def decklist_to_df(decklists):
    decklists = pd.DataFrame.from_records(decklists)
    decklists.index = [re.search(r"(\d+)", name).group(1) for name in decklists[0]]

    participants = decklists.iloc[:,0:5].copy()
    participants = participants.rename(columns={0:"Name", 1:"Event Date", 2:"Event Name", 3:"Format", 4:"Hero"})

    participants.index.rename("Gem ID", inplace=True) 

    decklists = decklists.drop([0,1,2,3,4], axis=1)

    decklists = pd.melt(decklists, ignore_index=False, value_name="import name")["import name"].to_frame()

    decklists[["Copies","Card"]] = decklists["import name"].str.split(" x ", expand=True)
    decklists = decklists.drop("import name", axis=1).dropna()

    decklists.index.rename("Gem ID", inplace=True) 

    return participants, decklists

def df_to_sql(dataframe, sql_table_name, index=True):
    connection = sqlite3.connect("pt_london.db")

    dataframe.to_sql(sql_table_name, connection, if_exists="replace", index=index)

In [146]:
pairings = get_pairings(pt_london_2025_results_url, rounds)
player_list = make_player_list(pairings)

decklists = get_decklist(player_list, pt_london_2025_decklists_url)
heroes = make_hero_list(decklists)

Round: 1 Table: 38 had a draw
Round: 1 Table: 38 had a draw
Round: 1 Table: 131 had a draw
Round: 1 Table: 131 had a draw
Round: 2 Table: 14 had a draw
Round: 2 Table: 14 had a draw
Round: 2 Table: 115 had a draw
Round: 2 Table: 115 had a draw
Round: 2 Table: 138 had a draw
Round: 2 Table: 138 had a draw
Round: 2 Table: 161 had a draw
Round: 2 Table: 161 had a draw
Round: 2 Table: 170 had a draw
Round: 2 Table: 170 had a draw
Round: 2 Table: 215 had a draw
Round: 2 Table: 215 had a draw
Round: 2 Table: 249 had a draw
Round: 2 Table: 249 had a draw
Round: 3 Table: 14 had a draw
Round: 3 Table: 14 had a draw
Round: 3 Table: 25 had a draw
Round: 3 Table: 25 had a draw
Round: 3 Table: 117 had a draw
Round: 3 Table: 117 had a draw
Round: 4 Table: 139 had a draw
Round: 4 Table: 139 had a draw
Round: 4 Table: 167 had a draw
Round: 4 Table: 167 had a draw
Round: 4 Table: 179 had a draw
Round: 4 Table: 179 had a draw
Round: 5 Table: 238 had a draw
Round: 5 Table: 238 had a draw
Round: 5 Table: 

In [148]:
tournament = decklist_to_df(decklists)
pairings_df = pd.DataFrame.from_records(pairings, columns=["Round","Table","Seat","Gem ID","Outcome"])

heroes_df = pd.Series(heroes)
heroes_df = heroes_df.to_frame()

heroes_df.index.rename("Hero ID", inplace=True)

participant_df = tournament[0]
decklist_df = tournament[1]

df_to_sql(participant_df, "participants")
df_to_sql(decklist_df, "decklists")
df_to_sql(pairings_df, "pairings", False)
df_to_sql(heroes_df, "heroes")